# Data Preprocessing - Parte 4 (Maps)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def load_data(file_path, sep=',', encoding='utf-8'):
    """Load data from a CSV file into a pandas DataFrame."""
    try:
        data = pd.read_csv(file_path, sep=sep, encoding=encoding)
        print("Data loaded successfully.")
        return data
    except Exception as e:
        print(f"An error occurred while loading the data: {e}")
        return None

def summarize_data(data):
    """Generate summary statistics of the DataFrame."""
    if data is not None:
        summary = data.describe()
        print("Data summary:")
        print(summary)
        return summary
    else:
        print("No data to summarize.")
        return None

## Choropleth Map based on the ACLED Data

In [3]:
data = load_data("../data/apac_data.csv", sep=';')
summary = summarize_data(data)

Data loaded successfully.
Data summary:
              EVENTS     FATALITIES  POPULATION_EXPOSURE             ID
count  197730.000000  197730.000000         9.976600e+04  197730.000000
mean        3.459632       2.036479         9.523680e+04    1538.280291
std         7.029385       9.795868         2.026675e+05     993.437542
min         1.000000       0.000000         1.000000e+00       1.000000
25%         1.000000       0.000000         1.054400e+04     593.000000
50%         1.000000       0.000000         3.482700e+04    1314.000000
75%         3.000000       0.000000         9.525400e+04    2283.000000
max       190.000000     618.000000         5.030356e+06    4157.000000


In [4]:
data.head()

,WEEK,REGION,COUNTRY,ADMIN1,EVENT_TYPE,SUB_EVENT_TYPE,EVENTS,FATALITIES,POPULATION_EXPOSURE,DISORDER_TYPE,ID,CENTROID_LATITUDE,CENTROID_LONGITUDE
0,31-dicembre-2016,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,6,15,NaN,Political violence,1,"36,966","73,3417"
1,07-gennaio-2017,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,5,28,NaN,Political violence,1,"36,966","73,3417"
2,21-gennaio-2017,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,1,4,NaN,Political violence,1,"36,966","73,3417"
3,04-febbraio-2017,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,1,4,NaN,Political violence,1,"36,966","73,3417"
4,11-febbraio-2017,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,4,41,NaN,Political violence,1,"36,966","73,3417"


In [5]:
# Filtering for Afghanistan
data['YEAR'] = data['WEEK'].apply(lambda x: int(x.split('-')[2]))

afg_data = data[data['COUNTRY'] == 'Afghanistan']

# Grouping by Admin1, Event_type and summing Events
grouped_data = afg_data.groupby(['ADMIN1', 'EVENT_TYPE', 'CENTROID_LATITUDE', 'CENTROID_LONGITUDE']).agg({'EVENTS': 'sum'}).reset_index()

grouped_data['ADMIN1'].value_counts()

ADMIN1
Badakhshan    6
Badghis       6
Wardak        6
Urozgan       6
Takhar        6
Samangan      6
Parwan        6
Panjshir      6
Paktika       6
Paktia        6
Nuristan      6
Nimruz        6
Nangarhar     6
Logar         6
Laghman       6
Kunduz        6
Kunar         6
Khost         6
Kapisa        6
Baghlan       6
Balkh         6
Bamyan        6
Daykundi      6
Farah         6
Faryab        6
Ghazni        6
Ghor          6
Helmand       6
Herat         6
Jowzjan       6
Kabul         6
Kandahar      6
Zabul         6
Sar-e Pol     5
Name: count, dtype: int64

In [6]:
grouped_data.head()

,ADMIN1,EVENT_TYPE,CENTROID_LATITUDE,CENTROID_LONGITUDE,EVENTS
0,Badakhshan,Battles,"36,966","73,3417",842
1,Badakhshan,Explosions/Remote violence,"36,966","73,3417",233
2,Badakhshan,Protests,"36,966","73,3417",49
3,Badakhshan,Riots,"36,966","73,3417",15
4,Badakhshan,Strategic developments,"36,966","73,3417",74


In [7]:
# For each ADMIN1, find the EVENT_TYPE with the maximum EVENTS
max_events = grouped_data.loc[grouped_data.groupby('ADMIN1')['EVENTS'].idxmax()]

max_events.head()

max_event = max_events['EVENTS'].max()
print(f"Maximum number of events in a single ADMIN1: {max_event}")

min_event = 0

Maximum number of events in a single ADMIN1: 4119


In [8]:
# find centroid lat and long for Sar-e Pol to add missing entry
sarepol_data = afg_data[afg_data['ADMIN1'] == 'Sar-e Pol']
sarepol_lat = sarepol_data['CENTROID_LATITUDE'].iloc[0]
sarepol_lon = sarepol_data['CENTROID_LONGITUDE'].iloc[0]

print(f"Sar-e Pol centroid: Latitude {sarepol_lat}, Longitude {sarepol_lon}")

Sar-e Pol centroid: Latitude 35,7152, Longitude 66,088


In [9]:
# find a row in the afg_data with ADMIN1 'Sar-e Pol' and print its details
sarepol_row = data[data['ADMIN1'] == 'Sar-e Pol']
print(sarepol_row['CENTROID_LATITUDE'])
print(sarepol_row['CENTROID_LONGITUDE'])

21766    35,7152
21767    35,7152
21768    35,7152
21769    35,7152
21770    35,7152
          ...   
22153    35,7152
22154    35,7152
22155    35,7152
22156    35,7152
22157    35,7152
Name: CENTROID_LATITUDE, Length: 392, dtype: object
21766    66,088
21767    66,088
21768    66,088
21769    66,088
21770    66,088
          ...  
22153    66,088
22154    66,088
22155    66,088
22156    66,088
22157    66,088
Name: CENTROID_LONGITUDE, Length: 392, dtype: object


In [10]:
# Adding a new entry for ADMIN1 == 'Sar-e Pol' and EVENT_TYPE == 'Riots' and EVENTS == 0
if not ((grouped_data['ADMIN1'] == 'Sar-e Pol') & (grouped_data['EVENT_TYPE'] == 'Riots')).any():
    new_entry = pd.DataFrame({
        'ADMIN1': ['Sar-e Pol'],
        'EVENT_TYPE': ['Riots'],
        'EVENTS': [0],
        'NORMALIZED_EVENTS': [0.0],
        'CENTROID_LATITUDE': [sarepol_lat],
        'CENTROID_LONGITUDE': [sarepol_lon]
    })
    grouped_data = pd.concat([grouped_data, new_entry], ignore_index=True)


In [11]:
# Normalizing the EVENTS for color scaling
grouped_data['NORMALIZED_EVENTS'] = (grouped_data['EVENTS'] - min_event) / (max_event - min_event)
grouped_data.head()

,ADMIN1,EVENT_TYPE,CENTROID_LATITUDE,CENTROID_LONGITUDE,EVENTS,NORMALIZED_EVENTS
0,Badakhshan,Battles,"36,966","73,3417",842,0.204419
1,Badakhshan,Explosions/Remote violence,"36,966","73,3417",233,0.056567
2,Badakhshan,Protests,"36,966","73,3417",49,0.011896
3,Badakhshan,Riots,"36,966","73,3417",15,0.003642
4,Badakhshan,Strategic developments,"36,966","73,3417",74,0.017966


In [12]:
print(grouped_data[grouped_data['ADMIN1'] == 'Sar-e Pol'])

        ADMIN1                  EVENT_TYPE CENTROID_LATITUDE  \
174  Sar-e Pol                     Battles           35,7152   
175  Sar-e Pol  Explosions/Remote violence           35,7152   
176  Sar-e Pol                    Protests           35,7152   
177  Sar-e Pol      Strategic developments           35,7152   
178  Sar-e Pol  Violence against civilians           35,7152   
203  Sar-e Pol                       Riots           35,7152   

    CENTROID_LONGITUDE  EVENTS  NORMALIZED_EVENTS  
174             66,088     392           0.095169  
175             66,088     120           0.029133  
176             66,088       8           0.001942  
177             66,088      48           0.011653  
178             66,088      65           0.015781  
203             66,088       0           0.000000  


In [13]:
# Transforming CENTROID_LATITUDE and CENTROID_LONGITUDE to numeric (pay attention: now their format has a comma instead of a dot)
# So first we need to replace commas with dots
grouped_data['CENTROID_LATITUDE'] = grouped_data['CENTROID_LATITUDE'].str.replace(',', '.')
grouped_data['CENTROID_LONGITUDE'] = grouped_data['CENTROID_LONGITUDE'].str.replace(',', '.')
grouped_data['CENTROID_LATITUDE'] = pd.to_numeric(grouped_data['CENTROID_LATITUDE'], errors='coerce')
grouped_data['CENTROID_LONGITUDE'] = pd.to_numeric(grouped_data['CENTROID_LONGITUDE'], errors='coerce')

grouped_data.head()

,ADMIN1,EVENT_TYPE,CENTROID_LATITUDE,CENTROID_LONGITUDE,EVENTS,NORMALIZED_EVENTS
0,Badakhshan,Battles,36.966,73.3417,842,0.204419
1,Badakhshan,Explosions/Remote violence,36.966,73.3417,233,0.056567
2,Badakhshan,Protests,36.966,73.3417,49,0.011896
3,Badakhshan,Riots,36.966,73.3417,15,0.003642
4,Badakhshan,Strategic developments,36.966,73.3417,74,0.017966


In [14]:
grouped_data['EVENT_TYPE'].unique()

array(['Battles', 'Explosions/Remote violence', 'Protests', 'Riots',
       'Strategic developments', 'Violence against civilians'],
      dtype=object)

In [15]:
# Saving grouped_data to CSV
#grouped_data.to_csv("../data/section_4/afg_choropleth.csv", index=False)

In [16]:
# afg_data['ADMIN1'].unique()

## Migrations

1. Codici Geografici e Flag Umanitari
Questi campi descrivono il paese di origine ("origin") e quello di accoglienza ("asylum").

origin_location_code / asylum_location_code: Sono i codici ISO3 dei paesi (es. AFG = Afghanistan, ALB = Albania).

_has_hrp (es. origin_has_hrp): È un valore booleano (True/False). Indica se quel paese ha attivo un Humanitarian Response Plan (HRP), ovvero un piano di risposta umanitaria coordinato dall'ONU per quell'anno. È utile per capire se il paese è ufficialmente in una situazione di crisi riconosciuta.

_in_gho (es. origin_in_gho): Indica se il paese è incluso nel Global Humanitarian Overview (GHO), il rapporto annuale che stima i bisogni umanitari globali. Solitamente, se un paese ha un HRP, è anche nel GHO.

2. Il Gruppo di Popolazione (population_group)
Questa è la colonna più critica per capire chi sono queste persone. I codici derivano dagli standard UNHCR. Ecco i significati dei valori che compaiono nel tuo esempio e altri comuni:

REF (Refugees): Rifugiati riconosciuti (persone fuggite dal proprio paese per conflitti o persecuzioni).

ASY (Asylum seekers): Richiedenti asilo (persone che hanno chiesto protezione internazionale ma la cui domanda non è ancora stata valutata definitivamente).

OOC (Others of Concern): Altre persone di interesse ("Others of Concern"). È una categoria ampia che include persone che non rientrano perfettamente nelle definizioni legali di rifugiato o sfollato interno, ma che ricevono comunque protezione o assistenza dall'UNHCR.

IDP: Sfollati interni (persone fuggite ma rimaste entro i confini del proprio paese).

ROC: Persone in situazioni simili ai rifugiati (Refugee-like).

3. Dati Demografici e Temporali
gender: f (femmine), m (maschi), all (tutti/non disaggregato).

age_range: Fascia d'età (es. 0-4, 18-59). Quando trovi all, il dato si riferisce al totale senza distinzione di età.

population: Il numero di persone in quella specifica categoria (es. "Donne, 18-59 anni, Rifugiate dall'Afghanistan che si trovano in Argentina").

reference_period_start / _end: Il periodo a cui si riferisce il dato. Noterai che spesso copre l'intero anno (es. 2020-01-01 al 2020-12-31), poiché i dati ufficiali UNHCR sui rifugiati sono spesso rilasciati su base annuale.

Fonti Ufficiali
Per approfondire o verificare futuri codici, ecco le fonti tecniche dirette che definiscono questo schema:

HDX HAPI Documentation - Affected People: Spiega specificamente la risorsa "Refugees & Persons of Concern" e come viene costruita (fonte: UNHCR).

HDX HAPI Enums (Enumerations): È la "stele di rosetta" per decodificare colonne come population_group.

Link alla documentazione: hdx-hapi.readthedocs.io

Centre for Humanitarian Data: Il blog ufficiale ha pubblicato articoli sul rilascio di queste API ("Expanded Coverage and New Features") spiegando proprio l'aggiunta dei campi has_hrp e in_gho per facilitare il filtraggio dei paesi in crisi.

In [17]:
migration_data = load_data("../data/hdx_hapi_refugees_afg.csv")
migration_data.head()

Data loaded successfully.


,origin_location_code,origin_has_hrp,origin_in_gho,asylum_location_code,asylum_has_hrp,asylum_in_gho,population_group,gender,age_range,min_age,max_age,population,reference_period_start,reference_period_end
0,AFG,True,True,AFG,True,True,OOC,f,0-4,0.0,4.0,8751,2020-01-01,2020-12-31
1,AFG,True,True,AFG,True,True,OOC,f,5-11,5.0,11.0,8775,2020-01-01,2020-12-31
2,AFG,True,True,AFG,True,True,OOC,f,12-17,12.0,17.0,5828,2020-01-01,2020-12-31
3,AFG,True,True,AFG,True,True,OOC,f,18-59,18.0,59.0,18958,2020-01-01,2020-12-31
4,AFG,True,True,AFG,True,True,OOC,f,60+,60.0,NaN,587,2020-01-01,2020-12-31


In [41]:
migration_data['origin_location_code'].unique()

array(['AFG', 'BEN', 'IRN', 'PAK', 'TUR', 'BGD', 'TJK', 'IND', 'IRQ',
       'CHN'], dtype=object)

In [44]:
# Checking the rows that have origin_location_code different from 'AFG' and also asylum_location_code different from 'AFG'
#migration_data[(migration_data['origin_location_code'] != 'AFG') & (migration_data['asylum_location_code'] != 'AFG')]
migration_data[(migration_data['origin_location_code'] != 'AFG')]

,origin_location_code,origin_has_hrp,origin_in_gho,asylum_location_code,asylum_has_hrp,asylum_in_gho,population_group,gender,age_range,min_age,max_age,population,year
2145,BEN,False,False,AFG,True,True,ASY,f,0-4,0.0,4.0,0,2020
2146,BEN,False,False,AFG,True,True,ASY,f,5-11,5.0,11.0,5,2020
2147,BEN,False,False,AFG,True,True,ASY,f,12-17,12.0,17.0,0,2020
2148,BEN,False,False,AFG,True,True,ASY,f,18-59,18.0,59.0,0,2020
2149,BEN,False,False,AFG,True,True,ASY,f,60+,60.0,NaN,0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43922,IRQ,False,True,AFG,True,True,REF,m,12-17,12.0,17.0,0,2004
43923,IRQ,False,True,AFG,True,True,REF,m,18-59,18.0,59.0,0,2004
43924,IRQ,False,True,AFG,True,True,REF,m,60+,60.0,NaN,0,2004
43925,IRQ,False,True,AFG,True,True,REF,m,all,NaN,NaN,0,2004


In [ ]:
# Conversion dictionary for location codes to location names
location_code_to_name = {
    'AFG': 'Afghanistan',
    'ALB': 'Albania',
    'ARE': 'United Arab Emirates',
    'ARG': 'Argentina',
    'ARM': 'Armenia',
    'AUT': 'Austria',
    'AZE': 'Azerbaijan',
    'BEL': 'Belgium',
    'BEN': 'Benin',
    'BGD': 'Bangladesh',
    'BGR': 'Bulgaria',
    'BIH': 'Bosnia and Herzegovina',
    'BLR': 'Belarus',
    'BRA': 'Brazil',
    'CAN': 'Canada',
    'CHE': 'Switzerland',
    'CHL': 'Chile',
    'CHN': 'China',
    'CIV': "Côte d'Ivoire",
    'COL': 'Colombia',
    'CUB': 'Cuba',
    'CYM': 'Cayman Islands',
    'CYP': 'Cyprus',
    'CZE': 'Czechia',
    'DEU': 'Germany',
    'DNK': 'Denmark',
    'DOM': 'Dominican Republic',
    'DZA': 'Algeria',
    'ECU': 'Ecuador',
    'EGY': 'Egypt',
    'ESP': 'Spain',
    'EST': 'Estonia',
    'ETH': 'Ethiopia',
    'FIN': 'Finland',
    'FJI': 'Fiji',
    'FRA': 'France',
    'GBR': 'United Kingdom',
    'GEO': 'Georgia',
    'GHA': 'Ghana',
    'GIN': 'Guinea',
    'GMB': 'Gambia',
    'GRC': 'Greece',
    'GUY': 'Guyana',
    'HKG': 'Hong Kong',
    'HRV': 'Croatia',
    'HUN': 'Hungary',
    'IDN': 'Indonesia',
    'IND': 'India',
    'IRL': 'Ireland',
    'IRN': 'Iran (Islamic Republic of)',
    'IRQ': 'Iraq',
    'ISL': 'Iceland',
    'ITA': 'Italy',
    'JOR': 'Jordan',
    'JPN': 'Japan',
    'KAZ': 'Kazakhstan',
    'KEN': 'Kenya',
    'KGZ': 'Kyrgyzstan',
    'KHM': 'Cambodia',
    'KOR': 'Republic of Korea',
    'KWT': 'Kuwait',
    'LBN': 'Lebanon',
    'LBY': 'Libya',
    'LIE': 'Liechtenstein',
    'LKA': 'Sri Lanka',
    'LTU': 'Lithuania',
    'LUX': 'Luxembourg',
    'LVA': 'Latvia',
    'MAR': 'Morocco',
    'MDA': 'Moldova (Republic of)',
    'MDG': 'Madagascar',
    'MEX': 'Mexico',
    'MKD': 'North Macedonia',
    'MLT': 'Malta',
    'MNE': 'Montenegro',
    'MNG': 'Mongolia',
    'MRT': 'Mauritania',
    'MYS': 'Malaysia',
    'NGA': 'Nigeria',
    'NLD': 'Netherlands',
    'NOR': 'Norway',
    'NPL': 'Nepal',
    'NRU': 'Nauru',
    'NZL': 'New Zealand',
    'OMN': 'Oman',
    'PAK': 'Pakistan',
    'PAN': 'Panama',
    'PER': 'Peru',
    'PHL': 'Philippines',
    'PNG': 'Papua New Guinea',
    'POL': 'Poland',
    'PRT': 'Portugal',
    'PRY': 'Paraguay',
    'QAT': 'Qatar',
    'ROU': 'Romania',
    'RUS': 'Russian Federation',
    'RWA': 'Rwanda',
    'SAU': 'Saudi Arabia',
    'SDN': 'Sudan',
    'SEN': 'Senegal',
    'SRB': 'Serbia',
    'SSD': 'South Sudan',
    'SUR': 'Suriname',
    'SVK': 'Slovakia',
    'SVN': 'Slovenia',
    'SWE': 'Sweden',
    'SWZ': 'Eswatini',
    'SYR': 'Syrian Arab Republic',
    'TCD': 'Chad',
    'TGO': 'Togo',
    'THA': 'Thailand',
    'TJK': 'Tajikistan',
    'TKM': 'Turkmenistan',
    'TUN': 'Tunisia',
    'TUR': 'Turkey',
    'TZA': 'Tanzania (United Republic of)',
    'UGA': 'Uganda',
    'UKR': 'Ukraine',
    'URY': 'Uruguay',
    'USA': 'United States of America',
    'UZB': 'Uzbekistan',
    'VEN': 'Venezuela (Bolivarian Republic of)',
    'ZAF': 'South Africa',
    'ZMB': 'Zambia',
    'ZWE': 'Zimbabwe',
}
# Add more mappings as needed

In [36]:
# The full list of location codes can be found here:
full_location_codes = {
    'ABW':  'Aruba',
    'AFG':  'Afghanistan',
    'AGO':  'Angola',
    'AIA':  'Anguilla',
    'ALA':  'Åland Islands',
    'ALB':  'Albania',
    'AND':  'Andorra',
    'ARE':  'United Arab Emirates',
    'ARG':  'Argentina',
    'ARM':  'Armenia',
    'ASM':  'American Samoa',
    'ATA':  'Antarctica',
    'ATF':  'French Southern Territories',
    'ATG':  'Antigua and Barbuda',
    'AUS':  'Australia',
    'AUT':  'Austria',
    'AZE':  'Azerbaijan',
    'BDI':  'Burundi',
    'BEL':  'Belgium',
    'BEN':  'Benin',
    'BES':  'Bonaire, Sint Eustatius and Saba',
    'BFA':  'Burkina Faso',
    'BGD':  'Bangladesh',
    'BGR':  'Bulgaria',
    'BHR':  'Bahrain',
    'BHS':  'Bahamas',
    'BIH':  'Bosnia and Herzegovina',
    'BLM':  'Saint Barthélemy',
    'BLR':  'Belarus',
    'BLZ':  'Belize',
    'BMU':  'Bermuda',
    'BOL':  'Bolivia, Plurinational State of',
    'BRA':  'Brazil',
    'BRB':  'Barbados',
    'BRN':  'Brunei Darussalam',
    'BTN':  'Bhutan',
    'BVT':  'Bouvet Island',
    'BWA':  'Botswana',
    'CAF':  'Central African Republic',
    'CAN':  'Canada',
    'CCK':  'Cocos (Keeling) Islands',
    'CHE':  'Switzerland',
    'CHL':  'Chile',
    'CHN':  'China',
    'CIV':  "Côte d'Ivoire",
    'CMR':  'Cameroon',
    'COD':  'Congo, Democratic Republic of the',
    'COG':  'Congo',
    'COK':  'Cook Islands',
    'COL':  'Colombia',
    'COM':  'Comoros',
    'CPV':  'Cabo Verde',
    'CRI':  'Costa Rica',
    'CUB':  'Cuba',
    'CUW':  'Curaçao',
    'CXR':  'Christmas Island',
    'CYM':  'Cayman Islands',
    'CYP':  'Cyprus',
    'CZE':  'Czechia',
    'DEU':  'Germany',
    'DJI':  'Djibouti',
    'DMA':  'Dominica',
    'DNK':  'Denmark',
    'DOM':  'Dominican Republic',
    'DZA':  'Algeria',
    'ECU':  'Ecuador',
    'EGY':  'Egypt',
    'ERI':  'Eritrea',
    'ESH':  'Western Sahara',
    'ESP':  'Spain',
    'EST':  'Estonia',
    'ETH':  'Ethiopia',
    'FIN':  'Finland',
    'FJI':  'Fiji',
    'FLK':  'Falkland Islands (Malvinas)',
    'FRA':  'France',
    'FRO':  'Faroe Islands',
    'FSM':  'Micronesia, Federated States of',
    'GAB':  'Gabon',
    'GBR':  'United Kingdom of Great Britain and Northern Ireland',
    'GEO':  'Georgia',
    'GGY':  'Guernsey',
    'GHA':  'Ghana',
    'GIB':  'Gibraltar',
    'GIN':  'Guinea',
    'GLP':  'Guadeloupe',
    'GMB':  'Gambia',
    'GNB':  'Guinea-Bissau',
    'GNQ':  'Equatorial Guinea',
    'GRC':  'Greece',
    'GRD':  'Grenada',
    'GRL':  'Greenland',
    'GTM':  'Guatemala',
    'GUF':  'French Guiana',
    'GUM':  'Guam',
    'GUY':  'Guyana',
    'HKG':  'Hong Kong',
    'HMD':  'Heard Island and McDonald Islands',
    'HND':  'Honduras',
    'HRV':  'Croatia',
    'HTI':  'Haiti',
    'HUN':  'Hungary',
    'IDN':  'Indonesia',
    'IMN':  'Isle of Man',
    'IND':  'India',
    'IOT':  'British Indian Ocean Territory',
    'IRL':  'Ireland',
    'IRN':  'Iran, Islamic Republic of',
    'IRQ':  'Iraq',
    'ISL':  'Iceland',
    'ISR':  'Israel',
    'ITA':  'Italy',
    'JAM':  'Jamaica',
    'JEY':  'Jersey',
    'JOR':  'Jordan',
    'JPN':  'Japan',
    'KAZ':  'Kazakhstan',
    'KEN':  'Kenya',
    'KGZ':  'Kyrgyzstan',
    'KHM':  'Cambodia',
    'KIR':  'Kiribati',
    'KNA':  'Saint Kitts and Nevis',
    'KOR':  'Korea, Republic of',
    'KWT':  'Kuwait',
    'LAO':  "Lao People's Democratic Republic",
    'LBN':  'Lebanon',
    'LBR':  'Liberia',
    'LBY':  'Libya',
    'LCA':  'Saint Lucia',
    'LIE':  'Liechtenstein',
    'LKA':  'Sri Lanka',
    'LSO':  'Lesotho',
    'LTU':  'Lithuania',
    'LUX':  'Luxembourg',
    'LVA':  'Latvia',
    'MAC':  'Macao',
    'MAF':  'Saint Martin (French part)',
    'MAR':  'Morocco',
    'MCO':  'Monaco',
    'MDA':  'Moldova, Republic of',
    'MDG':  'Madagascar',
    'MDV':  'Maldives',
    'MEX':  'Mexico',
    'MHL':  'Marshall Islands',
    'MKD':  'North Macedonia',
    'MLI':  'Mali',
    'MLT':  'Malta',
    'MMR':  'Myanmar',
    'MNE':  'Montenegro',
    'MNG':  'Mongolia',
    'MNP':  'Northern Mariana Islands',
    'MOZ':  'Mozambique',
    'MRT':  'Mauritania',
    'MSR':  'Montserrat',
    'MTQ':  'Martinique',
    'MUS':  'Mauritius',
    'MWI':  'Malawi',
    'MYS':  'Malaysia',
    'MYT':  'Mayotte',
    'NAM':  'Namibia',
    'NCL':  'New Caledonia',
    'NER':  'Niger',
    'NFK':  'Norfolk Island',
    'NGA':  'Nigeria',
    'NIC':  'Nicaragua',
    'NIU':  'Niue',
    'NLD':  'Netherlands, Kingdom of the',
    'NOR':  'Norway',
    'NPL':  'Nepal',
    'NRU':  'Nauru',
    'NZL':  'New Zealand',
    'OMN':  'Oman',
    'PAK':  'Pakistan',
    'PAN':  'Panama',
    'PCN':  'Pitcairn',
    'PER':  'Peru',
    'PHL':  'Philippines',
    'PLW':  'Palau',
    'PNG':  'Papua New Guinea',
    'POL':  'Poland',
    'PRI':  'Puerto Rico',
    'PRK':  "Korea, Democratic People's Republic of",
    'PRT':  'Portugal',
    'PRY':  'Paraguay',
    'PSE':  'Palestine, State of',
    'PYF':  'French Polynesia',
    'QAT':  'Qatar',
    'REU':  'Réunion',
    'ROU':  'Romania',
    'RUS':  'Russian Federation',
    'RWA':  'Rwanda',
    'SAU':  'Saudi Arabia',
    'SDN':  'Sudan',
    'SEN':  'Senegal',
    'SGP':  'Singapore',
    'SGS':  'South Georgia and the South Sandwich Islands',
    'SHN':  'Saint Helena, Ascension and Tristan da Cunha',
    'SJM':  'Svalbard and Jan Mayen',
    'SLB':  'Solomon Islands',
    'SLE':  'Sierra Leone',
    'SLV':  'El Salvador',
    'SMR':  'San Marino',
    'SOM':  'Somalia',
    'SPM':  'Saint Pierre and Miquelon',
    'SRB':  'Serbia',
    'SSD':  'South Sudan',
    'STP':  'Sao Tome and Principe',
    'SUR':  'Suriname',
    'SVK':  'Slovakia',
    'SVN':  'Slovenia',
    'SWE':  'Sweden',
    'SWZ':  'Eswatini',
    'SXM':  'Sint Maarten (Dutch part)',
    'SYC':  'Seychelles',
    'SYR':  'Syrian Arab Republic',
    'TCA':  'Turks and Caicos Islands',
    'TCD':  'Chad',
    'TGO':  'Togo',
    'THA':  'Thailand',
    'TJK':  'Tajikistan',
    'TKL':  'Tokelau',
    'TKM':  'Turkmenistan',
    'TLS':  'Timor-Leste',
    'TON':  'Tonga',
    'TTO':  'Trinidad and Tobago',
    'TUN':  'Tunisia',
    'TUR':  'Türkiye',
    'TUV':  'Tuvalu',
    'TWN':  'Taiwan, Province of China',
    'TZA':  'Tanzania, United Republic of',
    'UGA':  'Uganda',
    'UKR':  'Ukraine',
    'UMI':  'United States Minor Outlying Islands',
    'URY':  'Uruguay',
    'USA':  'United States of America',
    'UZB':  'Uzbekistan',
    'VAT':  'Holy See',
    'VCT':  'Saint Vincent and the Grenadines',
    'VEN':  'Venezuela, Bolivarian Republic of',
    'VGB':  'Virgin Islands (British)',
    'VIR':  'Virgin Islands (U.S.)',
    'VNM':  'Viet Nam',
    'VUT':  'Vanuatu',
    'WLF':  'Wallis and Futuna',
    'WSM':  'Samoa',
    'YEM':  'Yemen',
    'ZAF':  'South Africa',
    'ZMB':  'Zambia',
    'ZWE':  'Zimbabwe'
}


In [38]:
# Now checking if there are mismatches between my dictionary (location_code_to_name) and the full dictionary (full_location_codes)
for code in location_code_to_name.keys():
    if location_code_to_name[code] != full_location_codes.get(code, None):
        print(f"Mismatch for code {code}: {location_code_to_name[code]} != {full_location_codes.get(code, None)}")

Mismatch for code GBR: United Kingdom != United Kingdom of Great Britain and Northern Ireland
Mismatch for code IRN: Iran (Islamic Republic of) != Iran, Islamic Republic of
Mismatch for code KOR: Republic of Korea != Korea, Republic of
Mismatch for code MDA: Moldova (Republic of) != Moldova, Republic of
Mismatch for code NLD: Netherlands != Netherlands, Kingdom of the
Mismatch for code TUR: Turkey != Türkiye
Mismatch for code TZA: Tanzania (United Republic of) != Tanzania, United Republic of
Mismatch for code VEN: Venezuela (Bolivarian Republic of) != Venezuela, Bolivarian Republic of


In [40]:
## It is useless: there is the id feature in the json!

In [20]:
migration_data['reference_period_end'].unique()

array(['2020-12-31', '2021-12-31', '2022-12-31', '2023-12-31',
       '2024-12-31', '2015-12-31', '2016-12-31', '2017-12-31',
       '2018-12-31', '2019-12-31', '2010-12-31', '2011-12-31',
       '2012-12-31', '2013-12-31', '2014-12-31', '2005-12-31',
       '2006-12-31', '2007-12-31', '2008-12-31', '2009-12-31',
       '2001-12-31', '2002-12-31', '2003-12-31', '2004-12-31'],
      dtype=object)

In [21]:
# Adding a 'year' column extracting year from 'reference_period_end' which is in format 'YYYY-MM-DD'
migration_data['year'] = migration_data['reference_period_end'].apply(lambda x: int(x.split('-')[0]) if pd.notnull(x) else np.nan)
migration_data.head()

,origin_location_code,origin_has_hrp,origin_in_gho,asylum_location_code,asylum_has_hrp,asylum_in_gho,population_group,gender,age_range,min_age,max_age,population,reference_period_start,reference_period_end,year
0,AFG,True,True,AFG,True,True,OOC,f,0-4,0.0,4.0,8751,2020-01-01,2020-12-31,2020
1,AFG,True,True,AFG,True,True,OOC,f,5-11,5.0,11.0,8775,2020-01-01,2020-12-31,2020
2,AFG,True,True,AFG,True,True,OOC,f,12-17,12.0,17.0,5828,2020-01-01,2020-12-31,2020
3,AFG,True,True,AFG,True,True,OOC,f,18-59,18.0,59.0,18958,2020-01-01,2020-12-31,2020
4,AFG,True,True,AFG,True,True,OOC,f,60+,60.0,NaN,587,2020-01-01,2020-12-31,2020


In [22]:
# dropping the columns 'reference_period_start' and 'reference_period_end' as they are no longer needed
migration_data = migration_data.drop(columns=['reference_period_start', 'reference_period_end'])
migration_data.head()

,origin_location_code,origin_has_hrp,origin_in_gho,asylum_location_code,asylum_has_hrp,asylum_in_gho,population_group,gender,age_range,min_age,max_age,population,year
0,AFG,True,True,AFG,True,True,OOC,f,0-4,0.0,4.0,8751,2020
1,AFG,True,True,AFG,True,True,OOC,f,5-11,5.0,11.0,8775,2020
2,AFG,True,True,AFG,True,True,OOC,f,12-17,12.0,17.0,5828,2020
3,AFG,True,True,AFG,True,True,OOC,f,18-59,18.0,59.0,18958,2020
4,AFG,True,True,AFG,True,True,OOC,f,60+,60.0,NaN,587,2020


In [25]:
# veryfing if the same 'origin_location_code' can have different 'origin_has_hrp' values [NON è così]
migration_data.groupby('origin_location_code')['origin_in_gho'].nunique()

origin_location_code
AFG    1
BEN    1
BGD    1
CHN    1
IND    1
IRN    1
IRQ    1
PAK    1
TJK    1
TUR    1
Name: origin_in_gho, dtype: int64

In [31]:
print(np.any(migration_data.groupby('asylum_location_code')['asylum_has_hrp'].nunique() > 1))
print(np.any(migration_data.groupby('asylum_location_code')['asylum_in_gho'].nunique() > 1))

False
False


In [ ]:
# Now I can make various multiple datasets. Each of them should have the year, origin_location_code, asylum_location_code

# In the first one I get the total population of refugees for each year, origin_location_code and asylum_location_code
migration_year_cumulative = migration_data.groupby(['year', 'origin_location_code', 'asylum_location_code']).agg({'population': 'sum'}).reset_index()
migration_year_cumulative.head()
# Saving migration_year_cumulative to CSV
#migration_year_cumulative.to_csv("../data/section_4/migration_year_cumulative.csv", index=False)

In [48]:
# In the second one I do as before but also grouping by gender
migration_year_gender = migration_data.groupby(['year', 'origin_location_code', 'asylum_location_code', 'gender']).agg({'population': 'sum'}).reset_index()
migration_year_gender.head(10)

,year,origin_location_code,asylum_location_code,gender,population
0,2001,AFG,ARE,all,22
1,2001,AFG,ARE,f,12
2,2001,AFG,ARE,m,32
3,2001,AFG,AUS,all,6714
4,2001,AFG,AUS,f,0
5,2001,AFG,AUS,m,0
6,2001,AFG,AUT,all,1049
7,2001,AFG,AUT,f,0
8,2001,AFG,AUT,m,0
9,2001,AFG,AZE,all,1174


In [49]:
# Fixing some inconsistencies: the rows with the same year, origin_location_code, asylum_location_code should have the population number for the 'all' value of the gender column >= the sum of 'f' and 'm'
# When this is not the case, I set the 'all' value to be equal to the sum of 'f' and 'm'

# Saving it to migration_year_gender_fixed
migration_year_gender_fixed = migration_year_gender.copy()
for (year, origin, asylum), group in migration_year_gender_fixed.groupby(['year', 'origin_location_code', 'asylum_location_code']):
    total_population = group[group['gender'] == 'all']['population'].values
    if len(total_population) == 0:
        continue
    sum_f_m = group[group['gender'].isin(['f', 'm'])]['population'].sum()
    if total_population[0] < sum_f_m:
        migration_year_gender_fixed.loc[(migration_year_gender_fixed['year'] == year) & (migration_year_gender_fixed['origin_location_code'] == origin) & (migration_year_gender_fixed['asylum_location_code'] == asylum) & (migration_year_gender_fixed['gender'] == 'all'), 'population'] = sum_f_m


,year,origin_location_code,asylum_location_code,gender,population
0,2001,AFG,ARE,all,22
1,2001,AFG,ARE,f,12
2,2001,AFG,ARE,m,32
3,2001,AFG,AUS,all,6714
4,2001,AFG,AUS,f,0
5,2001,AFG,AUS,m,0
6,2001,AFG,AUT,all,1049
7,2001,AFG,AUT,f,0
8,2001,AFG,AUT,m,0
9,2001,AFG,AZE,all,1174


In [ ]:
migration_year_gender_fixed.head(10)
# Saving migration_year_gender_fixed to CSV
#migration_year_gender_fixed.to_csv("../data/section_4/migration_year_gender.csv", index=False)

In [53]:
# Now grouping by age_range as well
migration_year_age = migration_data.groupby(['year', 'origin_location_code', 'asylum_location_code', 'age_range']).agg({'population': 'sum'}).reset_index()
migration_year_age.head(10)

,year,origin_location_code,asylum_location_code,age_range,population
0,2001,AFG,ARE,0-4,0
1,2001,AFG,ARE,12-17,12
2,2001,AFG,ARE,18-59,5
3,2001,AFG,ARE,5-11,5
4,2001,AFG,ARE,60+,0
5,2001,AFG,ARE,all,44
6,2001,AFG,AUS,0-4,0
7,2001,AFG,AUS,12-17,0
8,2001,AFG,AUS,18-59,0
9,2001,AFG,AUS,5-11,0


In [54]:
migration_data['age_range'].unique()

array(['0-4', '5-11', '12-17', '18-59', '60+', 'all'], dtype=object)

In [55]:
migration_year_age_fixed = migration_year_age.copy()
for (year, origin, asylum), group in migration_year_age_fixed.groupby(['year', 'origin_location_code', 'asylum_location_code']):
    total_population = group[group['age_range'] == 'all']['population'].values
    if len(total_population) == 0:
        continue
    sum_f_m = group[group['age_range'].isin(['0-4', '5-11', '12-17', '18-59', '60+'])]['population'].sum()
    if total_population[0] < sum_f_m:
        migration_year_age_fixed.loc[(migration_year_age_fixed['year'] == year) & (migration_year_age_fixed['origin_location_code'] == origin) & (migration_year_age_fixed['asylum_location_code'] == asylum) & (migration_year_age_fixed['age_range'] == 'all'), 'population'] = sum_f_m

In [56]:
migration_year_age.head(10)

,year,origin_location_code,asylum_location_code,age_range,population
0,2001,AFG,ARE,0-4,0
1,2001,AFG,ARE,12-17,12
2,2001,AFG,ARE,18-59,5
3,2001,AFG,ARE,5-11,5
4,2001,AFG,ARE,60+,0
5,2001,AFG,ARE,all,44
6,2001,AFG,AUS,0-4,0
7,2001,AFG,AUS,12-17,0
8,2001,AFG,AUS,18-59,0
9,2001,AFG,AUS,5-11,0


In [57]:
migration_year_age_fixed.head(10)

,year,origin_location_code,asylum_location_code,age_range,population
0,2001,AFG,ARE,0-4,0
1,2001,AFG,ARE,12-17,12
2,2001,AFG,ARE,18-59,5
3,2001,AFG,ARE,5-11,5
4,2001,AFG,ARE,60+,0
5,2001,AFG,ARE,all,44
6,2001,AFG,AUS,0-4,0
7,2001,AFG,AUS,12-17,0
8,2001,AFG,AUS,18-59,0
9,2001,AFG,AUS,5-11,0


In [58]:
# find rows that are different between migration_year_age and migration_year_age_fixed
diff_rows = migration_year_age[migration_year_age_fixed['population'] != migration_year_age['population']]
print(diff_rows)

Empty DataFrame
Columns: [year, origin_location_code, asylum_location_code, age_range, population]
Index: []


In [ ]:
# saving migration_year_age_fixed to CSV
#migration_year_age_fixed.to_csv("../data/section_4/migration_year_age.csv", index=False)

In [60]:
# Now grouping by population_group as well
migration_data['population_group'].unique()

array(['OOC', 'ASY', 'REF', 'HST'], dtype=object)

In [61]:
# Grouping by year, origin_location_code, asylum_location_code, population_group
migration_year_population_group = migration_data.groupby(['year', 'origin_location_code', 'asylum_location_code', 'population_group']).agg({'population': 'sum'}).reset_index()
migration_year_population_group.head(10)

,year,origin_location_code,asylum_location_code,population_group,population
0,2001,AFG,ARE,ASY,48
1,2001,AFG,ARE,REF,18
2,2001,AFG,AUS,ASY,452
3,2001,AFG,AUS,REF,6262
4,2001,AFG,AUT,REF,1049
5,2001,AFG,AZE,ASY,2793
6,2001,AFG,AZE,REF,729
7,2001,AFG,BEL,REF,714
8,2001,AFG,BGR,ASY,1869
9,2001,AFG,BGR,REF,1565


In [ ]:
# Saving migration_year_population_group to CSV
# migration_year_population_group.to_csv("../data/section_4/migration_year_population_group.csv", index=False)